In [3]:
include("poisson.jl")
include("operators.jl")
include("labelGrid.jl")
include("boundaryConditions.jl")
using NearestNeighbors
using Combinatorics

In [9]:
#define grid
num_spaces = 100
levelset = zeros(Float64, num_spaces+1, num_spaces+1)
boundary = zeros(Int8, num_spaces+1, num_spaces+1)
is_inside = zeros(Bool, num_spaces+1, num_spaces+1)

#generates all sets of x, y
x = vec(collect(Base.Iterators.product(Base.Iterators.repeated(1:num_spaces+1, 2)...)))

#make wave on grid
amplitude = 10
frequency = 3
num_surf_points = (num_spaces)*100
startingx = amplitude
xs = ones(Float64, num_surf_points)
ys = collect(LinRange(1, num_spaces+1, num_surf_points))

#add buffer before perturbation
starty, endy = Int(num_surf_points * 0.2), Int(num_surf_points * 0.8) 
xs[starty:endy] .= @. amplitude + 1 + amplitude * cos(1/(ys[endy] - ys[starty])*(ys[starty:endy]-ys[starty]) * frequency * 2 * pi - pi)

#add right hand boundary to list of boundary points
xs = vcat(xs, ones(Float64, num_surf_points) .+ num_spaces)
ys = vcat(ys, collect(LinRange(1, num_spaces+1, num_surf_points)))

#make KDTree of surface points
tree = KDTree(transpose(hcat(xs, ys)))

#define grid points
gridx = first.(x)
gridy = last.(x)
points = transpose(hcat(gridx, gridy))

#find distance of each point to the surface
_, distances = knn(tree, points, 1)

#set grid points to distance from surface
for i in 1:length(x)
    xcoord = x[i][1]
    ycoord = num_spaces+1 - (x[i][2] - 1)
    levelset[ycoord, xcoord] = distances[i][1]
end

#set all points inside the bump to "is inside"
for i in 1:Int(length(xs)/2)
    is_inside[num_spaces+1 - (round(Int, ys[i]) - 1), 1:floor(Int, xs[i])] .= true
end

#if inside the surface, set levelset to a negative value
levelset = ifelse.(is_inside, -levelset, levelset)

#define boundary points as those <= 0 in levelset
levelset[:,  end] .= 0
boundary = levelset .<= 0
boundary[:, end] .= 1

101×101 BitMatrix:
 1  0  0  0  0  0  0  0  0  0  0  0  0  …  0  0  0  0  0  0  0  0  0  0  0  1
 1  0  0  0  0  0  0  0  0  0  0  0  0     0  0  0  0  0  0  0  0  0  0  0  1
 1  0  0  0  0  0  0  0  0  0  0  0  0     0  0  0  0  0  0  0  0  0  0  0  1
 1  0  0  0  0  0  0  0  0  0  0  0  0     0  0  0  0  0  0  0  0  0  0  0  1
 1  0  0  0  0  0  0  0  0  0  0  0  0     0  0  0  0  0  0  0  0  0  0  0  1
 1  0  0  0  0  0  0  0  0  0  0  0  0  …  0  0  0  0  0  0  0  0  0  0  0  1
 1  0  0  0  0  0  0  0  0  0  0  0  0     0  0  0  0  0  0  0  0  0  0  0  1
 1  0  0  0  0  0  0  0  0  0  0  0  0     0  0  0  0  0  0  0  0  0  0  0  1
 1  0  0  0  0  0  0  0  0  0  0  0  0     0  0  0  0  0  0  0  0  0  0  0  1
 1  0  0  0  0  0  0  0  0  0  0  0  0     0  0  0  0  0  0  0  0  0  0  0  1
 ⋮              ⋮              ⋮        ⋱     ⋮              ⋮              ⋮
 1  0  0  0  0  0  0  0  0  0  0  0  0     0  0  0  0  0  0  0  0  0  0  0  1
 1  0  0  0  0  0  0  0  0  0  0  0  0     0 

In [10]:
#parameters
delta = 0.1
epsilon = 0.05
v = 0.1
kf = 2E-6 * 1E-4/(2 * 1E-9)
kb = 0
Cb = 3.796E-6
dx = 2/num_spaces

#find which cells are next to electrode
boundary_label, boundary_indices = parseGrid(boundary)

#define normal vector at boudary cells
nx, ny = norms(levelset, boundary_indices)

#initialize fields
c = ones(Float64, num_spaces+1, num_spaces+1)
rho = zeros(Float64, num_spaces+1, num_spaces+1)
phi = zeros(Float64, num_spaces+1, num_spaces+1)

#define poisson matrix
A_matrix = assemblePoissonA(num_spaces, nx, ny)

10201×10201 SparseMatrixCSC{Float64, Int64} with 47361 stored entries:
⎡⠳⣦⡀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⎤
⎢⠀⠈⠻⣦⡀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⎥
⎢⠀⠀⠀⠈⠻⣦⡀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⎥
⎢⠀⠀⠀⠀⠀⠈⠻⣦⡀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⎥
⎢⠀⠀⠀⠀⠀⠀⠀⠈⠻⣦⡀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⎥
⎢⠀⠀⠀⠀⠀⠀⠀⠀⠀⠈⠻⣦⡀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⎥
⎢⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠈⠻⣦⡀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⎥
⎢⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠈⠻⣦⡀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⎥
⎢⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠈⠻⣦⡀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⎥
⎢⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠈⠻⣦⡀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⎥
⎢⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠈⠻⣦⡀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⎥
⎢⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠈⠻⣦⡀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⎥
⎢⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠈⠻⣦⡀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⎥
⎢⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠈⠻⣦⡀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⎥
⎢⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠈⠻⣦⡀⠀⠀⠀⠀⠀⠀⠀⠀⠀⎥
⎢⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠈⠻⣦⡀⠀⠀⠀⠀⠀⠀⠀⎥
⎢⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠈⠻⣦⡀⠀⠀⠀⠀⠀⎥
⎢⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠈⠻⣦⡀⠀⠀⠀⎥
⎢⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠈⠻⣦⡀⠀⎥
⎣⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠈⠻⢆⎦

In [11]:
dt = 10^-3
Rs = []
for time in 1:1000
    #solve poissons equation
    b = assemblePoissonb(num_spaces, rho)
    phi = reshape(A_matrix \ b, num_spaces+1, num_spaces+1)

    #enforcing no flux condition at boundary for all electrode-boundary cells
    #c, rho, Rs = enforceBV(c, rho, phi, nx, ny, boundary_indices)
    c, rho = enforceNoFlux(c, rho, phi, nx, ny, boundary_indices)
    
    #update c and rho; boundary is taken care of at the top of the loop
    grad2c = grad2(c)
    grad2rho = grad2(rho)
    grad2phi = grad2(phi)

    dcdt = epsilon .* (grad2c .+ graddotgrad(rho, phi) .+ (rho .* grad2phi))
    drhodt = epsilon .* (grad2rho .+ graddotgrad(c, phi) .+ (c .* grad2phi))
    c .+= (dcdt .* dt)
    rho .+= (drhodt .* dt)
end

In [12]:
using PyPlot
fig, ax = subplots(figsize=(7,7)) 
pos = ax.imshow(phi)
fig.colorbar(pos, ax=ax)
#scatter(xs .- 1, ys .- 1, s=.01)
ylabel("y")
xlabel("x")
title("phi")
savefig("/Users/arun/Desktop/Plots/phi.png", dpi=300)

In [13]:
using PyPlot
fig, ax = subplots(figsize=(7,7)) 
plot(collect(LinRange(-1, 1, num_spaces+1)), phi[50, :], label="middle cutline")
plot(collect(LinRange(-1, 1, num_spaces+1)), phi[1, :], label="edge cutline")
ylabel("y")
xlabel("x")
title("phi")
legend()
savefig("/Users/arun/Desktop/Plots/phicutlines.png", dpi=300)

In [9]:
reaction_rate_map = zeros(Float64, num_spaces+1, num_spaces+1)
for index in 1:length(b_ind)
    i = b_ind[index][1]
    j = b_ind[index][2]
    reaction_rate_map[i, j] = Rs[index]
    levelset[i, j] += Rs[index]
end